In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## ARES XML Processing Functions & Common params

In [70]:
from lxml import etree as ET
import pandas as pd
import re

# input parameters
xml_namespaces = {'are': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/ares/ares_answer_or/v_1.0.2', 
              'dtt': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/ares/ares_datatypes/v_1.0.2',
              'udt': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/uvis_datatypes/v_1.0.1',
              'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
             }
xml_data = ET.parse("darv_or.xml")

# Check if tag/attribute @ exists and return its value
def set_tag_value(attr_xpath, elem, xml_namespaces):
    if re.search('@',attr_xpath) is not None and elem.xpath(attr_xpath,namespaces = xml_namespaces) != []:# check for @ for attributes
        tag_value = elem.xpath(attr_xpath,namespaces = xml_namespaces)[0]
    elif elem.xpath(attr_xpath,namespaces = xml_namespaces) != []:
            tag_value = elem.xpath(attr_xpath,namespaces = xml_namespaces)[0].text
    else:
        tag_value = None
    return tag_value

# Iterate elements under parent tag and get element tag/attr values, static key values are assigned as well
def populate_table (xml_data,parent_tag, table_columns):
    table_columns_pd = pd.DataFrame(table_columns,columns =['tag','xpath','type'])
    iter_tag_values = []
    key_values = []
    for item in table_columns:
        if item[2] == 'S':
            key_values.append(set_tag_value(item[1],xml_data, xml_namespaces))
    if parent_tag:
        for elem in parent_tag:
            tag_values = []
            for item in table_columns:
                if item[2] == 'I':
                    tag_values.append(set_tag_value(item[1],elem, xml_namespaces)) 
            tag_values.insert(0,key_values)
            iter_tag_values.append(tag_values)
    else:
        iter_tag_values.insert(0,key_values)
    table_pd = pd.DataFrame(iter_tag_values, columns=table_columns_pd['tag'].tolist())
    return table_pd

### zakladni_udaje

In [74]:
basic_info_table_columns = [
    # Vypis Vypis_OR/Uvod
    ['Aktualizace_DB','./are:Odpoved/dtt:Vypis_OR/dtt:Uvod/dtt:Aktualizace_DB','S'],
    ['Datum_vypisu','./are:Odpoved/dtt:Vypis_OR/dtt:Uvod/dtt:Datum_vypisu','S'],
    # Zakladni_udaje
    ['ICO','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:ICO','S'],
    ['Stav_subjektu','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Stav/dtt:Stav_subjektu','S'],
    ['Konkurz','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Stav/dtt:Konkurz/dtt:Kod','S'],
    ['Vyrovnani','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Stav/dtt:Vyrovnani/dtt:Kod','S'],
    ['Zamitnuti','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Stav/dtt:Zamitnuti/dtt:Kod','S'],
    ['Likvidace','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Stav/dtt:Likvidace/dtt:Kod','S'],
    ['Datum_zapisu_OR','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Datum_zapisu_OR','S'],
    # Registrace
    
    ['Soud_Kod','./are:Odpoved/dtt:Vypis_OR/dtt:Registrace/dtt:Spisova_znacka/dtt:Soud/dtt:Kod','S'],
    ['Soud_Text','./are:Odpoved/dtt:Vypis_OR/dtt:Registrace/dtt:Spisova_znacka/dtt:Soud/dtt:Text','S'],
    ['Oddil_vlozka','./are:Odpoved/dtt:Vypis_OR/dtt:Registrace/dtt:Spisova_znacka/dtt:Oddil_vlozka','S'],
    
    # Pravni_forma_OR
    ['Kod_PF','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Pravni_forma_OR/dtt:Kod_PF','S'],
    ['Nazev_PF','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Pravni_forma_OR/dtt:Nazev_PF','S'],
    ['PF_osoba','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Pravni_forma_OR/dtt:PF_osoba','S'],
    ['TZU_osoba','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Pravni_forma_OR/dtt:TZU_osoba','S'],
    
]
populate_table(xml_data,None,basic_info_table_columns)

,Aktualizace_DB,Datum_vypisu,ICO,Stav_subjektu,Konkurz,Vyrovnani,Zamitnuti,Likvidace,Datum_zapisu_OR,Soud_Kod,Soud_Text,Oddil_vlozka,Kod_PF,Nazev_PF,PF_osoba,TZU_osoba
0,2018-08-22,2018-08-23,63275635,Aktivní,0,0,0,0,1995-08-01,3,Krajský soud v Českých Budějovicích,B 698,121,Akciová společnost,P,Tuzemská


### obchodni_firma

In [80]:
company_name_table_columns = [
    ['ICO','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:ICO','S'],
    ['dod','./@dod','I'],
    ['ddo','./@ddo','I'],
    ['Obchodni_firma','.','I']
]
company_name_parent_tag = xml_data.xpath('./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Obchodni_firma',namespaces = xml_namespaces)
populate_table(xml_data,company_name_parent_tag,company_name_table_columns)

,ICO,dod,ddo,Obchodni_firma
0,[63275635],2006-06-30,None,MADETA a. s.
1,[63275635],2005-07-04,2006-06-30,MADETA Group a. s.
2,[63275635],1995-08-01,2005-07-04,"JIHOMILK, akciová společnost"


### sidlo

In [59]:
# S = static record key from ancestor tag (static), I = iterative records under parent_tag 
headquarters_table_columns = [
    ['ICO','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:ICO','S'],
    ['dod','./@dod','I'],
    ['ddo','./@ddo','I'],
    ['ID_adresy','./dtt:ID_adresy','I'],
    ['Kod_statu','./dtt:Kod_statu','I'],
    ['Nazev_statu','./dtt:Nazev_statu','I'],
    ['Nazev_okresu','./dtt:Nazev_okresu','I'],
    ['Nazev_obce','./dtt:Nazev_obce','I'],
    ['Nazev_casti_obce','./dtt:Nazev_casti_obce','I'],
    ['Nazev_ulice','./dtt:Nazev_ulice','I'],
    ['Cislo_domovni','./dtt:Cislo_domovni','I'],
    ['Typ_cislo_domovni','./dtt:Typ_cislo_domovni','I'],
    ['Cislo_orientacni','./dtt:Cislo_orientacni','I'],
    ['PSC','./dtt:PSC','I'],
    ['Cislo_do_adresy','./dtt:Cislo_do_adresy','I']
                      ]
headquarters_parent_tag = xml_data.xpath('./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Sidlo',namespaces = xml_namespaces)
populate_table(xml_data,headquarters_parent_tag,headquarters_table_columns)

,ICO,dod,ddo,ID_adresy,Kod_statu,Nazev_statu,Nazev_okresu,Nazev_obce,Nazev_casti_obce,Nazev_ulice,Cislo_domovni,Typ_cislo_domovni,Cislo_orientacni,PSC,Cislo_do_adresy
0,[63275635],2017-11-23,None,29643147,203,Česká republika,České Budějovice,České Budějovice,České Budějovice 4,Rudolfovská tř.,246,1,83,37001,None
1,[63275635],2016-12-07,2017-11-23,29643145,203,Česká republika,České Budějovice,České Budějovice,České Budějovice 4,Rudolfovská tř.,246,1,83,37001,None
2,[63275635],2002-08-14,2016-12-07,29643143,203,Česká republika,České Budějovice,České Budějovice,None,Rudolfovská,None,None,None,37050,246/83
3,[63275635],1995-08-01,2002-08-14,29643141,203,Česká republika,None,České Budějovice,None,Rudolfovská,None,None,None,37050,83


### predmet_podnikani
__TODO__: 

Rozmyslet jestli brat i ostatni cinnosti (ucel nadace atp.), 

texty obsahuji \n nutno strip, nebo replace

In [79]:
cinnosti_table_columns = [
    ['ICO','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:ICO','S'],
    ['dod','./@dod','I'],
    ['ddo','./@ddo','I'],
    ['predmet_podnikani','.','I']
]
cinnosti_parent_tag = xml_data.xpath('./are:Odpoved/dtt:Vypis_OR/dtt:Cinnosti/dtt:Predmet_podnikani/dtt:Text',namespaces = xml_namespaces)
populate_table(xml_data,cinnosti_parent_tag,cinnosti_table_columns)

,ICO,dod,ddo,predmet_podnikani
0,[63275635],2014-08-06,None,"\nprodej kvasného lihu, konzumního lihu a liho..."
1,[63275635],2011-11-10,None,\nopravy silničních vozidel\n
2,[63275635],2011-11-10,None,\nklempířství a oprava karoserií\n
3,[63275635],2009-03-06,None,"\nzámečnictví, nástrojářství\n"
4,[63275635],2009-03-06,None,"\nvýroba, obchod a služby neuvedené v přílohác..."
5,[63275635],2009-03-06,None,"\nvýroba, instalace, opravy elektrických stroj..."
6,[63275635],2008-01-22,2009-03-06,"\ntestování, měření, analýzy a kontroly\n"
7,[63275635],2007-10-11,2015-04-18,\nkoupě za účelem jeho dalšího prodeje a prodej\n
8,[63275635],2007-10-11,2015-04-18,\nvelkoobchod\n
9,[63275635],2007-10-11,2015-04-18,\nzprostředkovatelská činnost\n


### statutarni_organ
__TODO:__ zkontrolovat v profilingu, existenci FO bez bydliste v takovem pripade jeste iterovat pres FO a nasledne spojit DFs pres unique

In [72]:
statutory_table_columns = [
    ['ICO','./are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:ICO','S'],
    ['Titul_pred','../dtt:Titul_pred','I'],
    ['Jmeno','../dtt:Jmeno','I'],
    ['Prijmeni','../dtt:Prijmeni','I'],
    ['Titul_za','../dtt:Titul_za','I'],
    ['Datum_narozeni','../dtt:Datum_narozeni','I'],
    ['Rodne_cislo','../dtt:Rodne_cislo','I'],
    ['Osoba_textem','../dtt:Osoba_textem','I'],
    ['dod','./@dod','I'],
    ['ddo','./@ddo','I'],
    ['ID_adresy','./dtt:ID_adresy','I'],
    ['Kod_statu','./dtt:Kod_statu','I'],
    ['Nazev_statu','./dtt:Nazev_statu','I'],
    ['Nazev_okresu','./dtt:Nazev_okresu','I'],
    ['Nazev_obce','./dtt:Nazev_obce','I'],
    ['Nazev_casti_obce','./dtt:Nazev_casti_obce','I'],
    ['Nazev_ulice','./dtt:Nazev_ulice','I'],
    ['Cislo_domovni','./dtt:Cislo_domovni','I'],
    ['Typ_cislo_domovni','./dtt:Typ_cislo_domovni','I'],
    ['Cislo_orientacni','./dtt:Cislo_orientacni','I'],
    ['PSC','./dtt:PSC','I'],
    ['Cislo_do_adresy','./dtt:Cislo_do_adresy','I']
]

statutory_parent_tag = xml_data.xpath('./are:Odpoved/dtt:Vypis_OR/dtt:Statutarni_organ/dtt:Clen_SO/dtt:Clen/dtt:Fyzicka_osoba/dtt:Bydliste',namespaces = xml_namespaces)
data = populate_table(xml_data,statutory_parent_tag,statutory_table_columns)
data.head(5)

,ICO,Titul_pred,Jmeno,Prijmeni,Titul_za,Datum_narozeni,Rodne_cislo,Osoba_textem,dod,ddo,...,Nazev_statu,Nazev_okresu,Nazev_obce,Nazev_casti_obce,Nazev_ulice,Cislo_domovni,Typ_cislo_domovni,Cislo_orientacni,PSC,Cislo_do_adresy
0,[63275635],Ing.,PAVEL,BURYÁNEK,None,1984-12-02,None,None,2017-09-13,None,...,Česká republika,Tábor,Veselí nad Lužnicí,Veselí nad Lužnicí II,Na Pasekách,948,1,None,39181,None
1,[63275635],Ing.,MILAN,TEPLÝ,None,1949-03-26,None,None,2016-12-07,None,...,Česká republika,České Budějovice,České Budějovice,České Budějovice 2,České Vrbné,2307,1,None,37011,None
2,[63275635],Ing.,PETR,PAYER,None,1976-07-17,None,None,2016-12-07,None,...,Česká republika,České Budějovice,České Budějovice,České Budějovice 3,Nerudova,904,1,66,37004,None
3,[63275635],Ing.,JAN,TEPLÝ,None,1980-03-15,None,None,2016-12-07,None,...,Česká republika,České Budějovice,České Budějovice,České Budějovice 2,České Vrbné,2307,1,None,37011,None
4,[63275635],Ing.,PAVEL,BURYÁNEK,None,1984-12-02,None,None,2016-12-07,2017-09-13,...,Česká republika,Tábor,Veselí nad Lužnicí,Veselí nad Lužnicí I,A. Jiráska,349,1,None,39181,None
